## Data

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns

# Load the Excel file to explore its content and structure
campaign_data = pd.read_csv('Amazon Ads All.csv')

# Display the first few rows of the dataframe and the column names to understand the dataset
campaign_data.head(), campaign_data.columns


(   ADV_ID    AD_ID         SKU     placement_slot  impressions  clicks  \
 0   84373  6370624  9985955750  offsite_mobileweb         1089       0   
 1   66528  3003509  6064714310  offsite_mobileapp            4       0   
 2   88613  9302679  7428981046    offsite_desktop            4       0   
 3   57975  3325755  7489730099    offsite_desktop           13       0   
 4   13386  6532070  6004552148  offsite_mobileweb          246       0   
 
    auction_cost  adjusted_cost hit_day_utc  \
 0         10.89          8.790    02/05/24   
 1          0.08          0.032    02/05/24   
 2          0.08          0.032    02/05/24   
 3          0.26          0.127    02/05/24   
 4          2.46          1.978    02/05/24   
 
                             targeting_secondary                 vertical  \
 0  category=Automotive Replacement Control Arms               Automotive   
 1              category=Canned & Jarred Seafood  Consumer Packaged Goods   
 2                       category

## IMPRESSIONS

Below code is used to see feature importance of impressions based on 'clicks', 'day_of_week', 'month', 'placement_slot_', targeting_secondary_
Influence of unknown verticals.

## Cleaning

In [13]:
## CLEANING
# Attempt to parse dates in the '%d/%m/%y' format
campaign_data['hit_day_utc_parsed'] = pd.to_datetime(campaign_data['hit_day_utc'], format='%d/%m/%y', errors='coerce')

# Parse remaining unparsed dates in the '%Y-%m-%d %H:%M:%S' format
mask = campaign_data['hit_day_utc_parsed'].isna()
campaign_data.loc[mask, 'hit_day_utc_parsed'] = pd.to_datetime(campaign_data.loc[mask, 'hit_day_utc'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Check for any remaining non-parsed dates
remaining_non_parsed = campaign_data[campaign_data['hit_day_utc_parsed'].isna()]
if not remaining_non_parsed.empty:
    print("Remaining non-parsed dates:")
    print(remaining_non_parsed)

# Print unique months in the parsed 'hit_day_utc_parsed' column
print("Unique months in 'hit_day_utc_parsed':")
print(campaign_data['hit_day_utc_parsed'].dt.month.unique())

# Extract month and day of the week from parsed dates
campaign_data['month'] = campaign_data['hit_day_utc_parsed'].dt.month
campaign_data['day_of_week'] = campaign_data['hit_day_utc_parsed'].dt.dayofweek

campaign_data['month_name'] = campaign_data['hit_day_utc_parsed'].dt.strftime('%B')  # Get month names
campaign_data['day_name'] = campaign_data['hit_day_utc_parsed'].dt.strftime('%A')    # Get day names

# Fill missing values in 'vertical' and 'sub_vertical' with 'Unknown'
campaign_data['vertical'].fillna('Unknown', inplace=True)
campaign_data['sub_vertical'].fillna('Unknown', inplace=True)

# Identify the last 6 columns
last_6_columns = campaign_data.columns[-6:]

# Fill missing values in the last 6 columns with 0
campaign_data[last_6_columns] = campaign_data[last_6_columns].fillna(0)

# Display the cleaned dataset
print("Cleaned Campaign Data Sample:")
print(campaign_data.head())

# Inspect the 'hit_day_utc' column to understand its format
print("Sample of 'hit_day_utc' column:")
print(campaign_data['hit_day_utc'].head(10))


# Prepare the dataset
# Create additional features
campaign_data['ctr'] = campaign_data['clicks'] / campaign_data['impressions']

# Display columns to ensure the correct ones are present
print("Columns in Campaign Data:")
print(campaign_data.columns)




Unique months in 'hit_day_utc_parsed':
[5 4 3 2]


/var/folders/80/ghdt2w012_x2yg018vx45m300000gn/T/ipykernel_76694/1913618617.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  campaign_data['vertical'].fillna('Unknown', inplace=True)
/var/folders/80/ghdt2w012_x2yg018vx45m300000gn/T/ipykernel_76694/1913618617.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting 

Cleaned Campaign Data Sample:
   ADV_ID    AD_ID         SKU     placement_slot  impressions  clicks  \
0   84373  6370624  9985955750  offsite_mobileweb         1089       0   
1   66528  3003509  6064714310  offsite_mobileapp            4       0   
2   88613  9302679  7428981046    offsite_desktop            4       0   
3   57975  3325755  7489730099    offsite_desktop           13       0   
4   13386  6532070  6004552148  offsite_mobileweb          246       0   

   auction_cost  adjusted_cost hit_day_utc  \
0         10.89          8.790    02/05/24   
1          0.08          0.032    02/05/24   
2          0.08          0.032    02/05/24   
3          0.26          0.127    02/05/24   
4          2.46          1.978    02/05/24   

                            targeting_secondary  ...  \
0  category=Automotive Replacement Control Arms  ...   
1              category=Canned & Jarred Seafood  ...   
2                       category=Comedy & Humor  ...   
3                  categ

##  Model 1

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


# Define features and target for adjusted cost
features_cost = ['clicks', 'targeting_secondary', 'placement_slot', 'month_name', 'day_name', 'vertical', 'sub_vertical']
target_cost = 'impressions'

# Prepare the data
X_cost = campaign_data[features_cost]
y_cost = campaign_data[target_cost]

# One-hot encode categorical features and standardize numerical features
preprocessor_cost = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), [ 'clicks']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['targeting_secondary', 'placement_slot', 'month_name', 'day_name', 'vertical', 'sub_vertical'])
    ])

# Define and train 
model_cost = Pipeline(steps=[
    ('preprocessor', preprocessor_cost),
    ('regressor', GradientBoostingRegressor(random_state=42))
])

X_train_cost, X_test_cost, y_train_cost, y_test_cost = train_test_split(X_cost, y_cost, test_size=0.2, random_state=42)
model_cost.fit(X_train_cost, y_train_cost)

# Predict and evaluate the adjusted cost model
y_pred_cost = model_cost.predict(X_test_cost)
mse_cost = mean_squared_error(y_test_cost, y_pred_cost)
mae_cost = mean_absolute_error(y_test_cost, y_pred_cost)
r2_cost = r2_score(y_test_cost, y_pred_cost)

# Display the evaluation metrics for adjusted cost model
print(f"Adjusted Cost Model - Mean Squared Error: {mse_cost}")
print(f"Adjusted Cost Model - Mean Absolute Error: {mae_cost}")
print(f"Adjusted Cost Model - R-squared: {r2_cost}")

# Get feature names after one-hot encoding
encoded_feature_names_cost = model_cost.named_steps['preprocessor'].transformers_[1][1].get_feature_names_out(['targeting_secondary', 'placement_slot', 'month_name', 'day_name', 
                                                                                                               'vertical', 'sub_vertical'])

# Combine original numerical features with encoded categorical features
all_features_cost = ['clicks'] + list(encoded_feature_names_cost)

# Calculate feature importance
importance_cost = model_cost.named_steps['regressor'].feature_importances_
feature_importance_cost = pd.DataFrame({
    'Feature': all_features_cost,
    'Importance': importance_cost
}).sort_values(by='Importance', ascending=False)

print("Feature Importance for Impressions Model:")
pd.set_option('display.max_rows', None)
print(feature_importance_cost)

Adjusted Cost Model - Mean Squared Error: 5092641.228560143
Adjusted Cost Model - Mean Absolute Error: 1252.093864708542
Adjusted Cost Model - R-squared: 0.6445386412026036
Feature Importance for Adjusted Cost Model:
                                                Feature    Importance
2495                   placement_slot_offsite_mobileapp  2.729774e-01
2402  targeting_secondary_category=Women's Running S...  2.347242e-01
0                                                clicks  1.860295e-01
894      targeting_secondary_category=Foundation Makeup  6.451624e-02
2486    targeting_secondary_content-category=Soap Opera  3.485759e-02
1922  targeting_secondary_category=Sheet & Pillowcas...  2.503835e-02
2494                     placement_slot_offsite_desktop  2.365550e-02
1955  targeting_secondary_category=Single-Serve Coff...  2.343398e-02
1268     targeting_secondary_category=Kids' Electronics  1.127454e-02
1126        targeting_secondary_category=Home & Kitchen  1.121712e-02
2043       ta

## Model 2

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


# Define features and target for adjusted cost
features_cost = ['clicks', 'targeting_secondary', 'placement_slot', 'month_name', 'day_name', 'vertical', 'sub_vertical']
target_cost = 'impressions'

# Prepare the data
X_cost = campaign_data[features_cost]
y_cost = campaign_data[target_cost]

# One-hot encode categorical features and standardize numerical features
preprocessor_cost = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), [ 'clicks']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['targeting_secondary', 'placement_slot', 'month_name', 'day_name', 'vertical', 'sub_vertical'])
    ])

# Define and train 
model_cost = Pipeline(steps=[
    ('preprocessor', preprocessor_cost),
    ('regressor', RandomForestRegressor(random_state=42))
])

X_train_cost, X_test_cost, y_train_cost, y_test_cost = train_test_split(X_cost, y_cost, test_size=0.2, random_state=42)
model_cost.fit(X_train_cost, y_train_cost)

# Predict and evaluate the adjusted cost model
y_pred_cost = model_cost.predict(X_test_cost)
mse_cost = mean_squared_error(y_test_cost, y_pred_cost)
mae_cost = mean_absolute_error(y_test_cost, y_pred_cost)
r2_cost = r2_score(y_test_cost, y_pred_cost)

# Display the evaluation metrics for adjusted cost model
print(f"Adjusted Cost Model - Mean Squared Error: {mse_cost}")
print(f"Adjusted Cost Model - Mean Absolute Error: {mae_cost}")
print(f"Adjusted Cost Model - R-squared: {r2_cost}")

# Get feature names after one-hot encoding
encoded_feature_names_cost = model_cost.named_steps['preprocessor'].transformers_[1][1].get_feature_names_out(['targeting_secondary', 'placement_slot', 'month_name', 'day_name', 
                                                                                                               'vertical', 'sub_vertical'])

# Combine original numerical features with encoded categorical features
all_features_cost = ['clicks'] + list(encoded_feature_names_cost)

# Calculate feature importance
importance_cost = model_cost.named_steps['regressor'].feature_importances_
feature_importance_cost = pd.DataFrame({
    'Feature': all_features_cost,
    'Importance': importance_cost
}).sort_values(by='Importance', ascending=False)

print("Feature Importance for Impressions Model:")
pd.set_option('display.max_rows', None)
print(feature_importance_cost)

Adjusted Cost Model - Mean Squared Error: 2288630.2707646866
Adjusted Cost Model - Mean Absolute Error: 548.6085956182906
Adjusted Cost Model - R-squared: 0.8402558536288491
Feature Importance for Impressions Model:
                                                Feature    Importance
2495                   placement_slot_offsite_mobileapp  1.809080e-01
2402  targeting_secondary_category=Women's Running S...  1.580341e-01
0                                                clicks  1.551999e-01
894      targeting_secondary_category=Foundation Makeup  5.622356e-02
2486    targeting_secondary_content-category=Soap Opera  2.712819e-02
2494                     placement_slot_offsite_desktop  2.624305e-02
1922  targeting_secondary_category=Sheet & Pillowcas...  2.326625e-02
1955  targeting_secondary_category=Single-Serve Coff...  1.938817e-02
2504                                    day_name_Sunday  1.537536e-02
2498                                month_name_February  1.518512e-02
1126        ta